In [23]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
from math import floor
import pickle
from pymongo import MongoClient
from os import listdir
from os.path import isfile, join, isdir
import time
from os import listdir
import os
import re
import pickle
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
import ast
class dbConf(object):
    # connect to database
    client = MongoClient('mongodb://localhost:27017/')
    try:
        mdb = client.racmo
        Notifications = mdb.Notifications
        PdfFiles=mdb.PdfFiles
        Keywords=mdb.Keywords
    except Exception as e:
        print(e)

PDF_DIR = "/home/thrymr/Racmo/processed/notification_samples_0904_02"


In [2]:
pdf_files = [f for f in listdir(PDF_DIR)\
                 if isfile(join(PDF_DIR,  f)) ]
ls=list()    
for pdf_file in pdf_files:
    ls.append(pdf_file.split('_'))
df=pd.DataFrame(ls)
# df1=df.groupby(3)
df = df[pd.notnull(df[3])]
fg=list(df.groupby(3))
ls=[]
for k,gdf in  fg:
    flist=[]
    elist=[]
    for i, row in gdf.iterrows():
        row=row.dropna()
        fln='_'.join(list(row))
        flist.append(fln)
        elist.append(fln[-3:].lower())
    fgroup={'group':k,'files':flist,'length':len(flist),'min_filename':min(flist, key=len),'extensions':elist}
    ls.append(fgroup)
flgdf=pd.DataFrame(ls)
flgdf=flgdf.dropna(thresh=1,axis=1)


In [1]:
def get_structured_files_dataframe(df):
    tmp = pd.DataFrame(columns=["file","group"])
    j = 0
    for i, r in df.iterrows():
        for f in r.files:
            tmp.loc[j] = [f, r.group]
            j = j+1
#     assert tmp.apply(lambda x:x.file.split("_")[3] == x.group, axis = 1).all()
    tmp["ext"] = tmp.apply(lambda x : x.file.split(".")[-1],axis =1)
    tmp["length"] = tmp.apply(lambda x : len(x.file),axis =1)
    TI = []
    OI = []
    for case, indices in tmp.groupby("group").groups.items():
        ticket_index = tmp.loc[indices].length.values.argmin()
        for i, idx in enumerate(indices):
            if i == ticket_index:
                TI.append(idx)
            else:
                OI.append(idx)

    tmp['type'] = "OTHER"
    tmp.loc[TI,"type"] = "TICKET"
    return tmp

# df=pd.read_csv('/home/thrymr/file_group.csv')
fdf = get_structured_files_dataframe(flgdf)
fdf=fdf.rename(columns={"file":"filename","group":"filegroup","type":"filetype"})


In [2]:
def df_to_json(df):
    js = {}
    vals = df[0].values
    ls = []
    for i, v in enumerate(vals):
        if not v:
            ls.append(ls[-1])
        else:
            ls.append(v)
    df[0] = ls
    for key, idx in df.groupby([0]).groups.items():
        
        js[key] = {}
        
        for val in df.loc[idx][[1,2]].values:
            if val[0] == "" and val[1] == "":
                js[key]["_value"] = ""
            elif val[0] == "" and val[1] != "":
                js[key]["_value"] = val[1]
            elif val[0] != "" and val[1] == "":
                js[key]["_value"] = val[0]
            elif val[0] != "" and val[1] != "":
                js[key][val[0]] = val[1]                
        if(key=="Documentos"):
            js[key] = df.loc[idx][[1,2]].values.tolist()
        if len(js[key]) == 1 and "_value" in js[key]:
            js[key] = js[key]["_value"]
    return js
def get_pdf_text_2(path):
    try:
        pdf=pdfquery.PDFQuery(path)
        pdf.load()
        pdftext=""
        pgn=len(pdf.tree.getroot().getchildren())
        for i in range(0,pgn):
            root = pdf.tree.getroot().getchildren()[i]
            for node in root.iter():
                try:
    #                 if node.tag == "LTTextLineHorizontal" or node.tag == "LTTextBox":
                    if node.text:
                        pdftext=pdftext+" "+node.text
    #                     pdftext=pdftext+"\n"
                except Exception as e:
                    print(node.tag, e)
        return pdftext
    except Exception as e:
        return('Error:'+str(e))
def get_rtf_text(path):
    print('unrtf --text '+path)
    text=os.popen('unrtf --text '+path).read()
    print(text)
    return text

def parse_ticket(pdf_path):
    _JSON = {"filepath" : pdf_path}
    try:
            pdf = pdfquery.PDFQuery(pdf_path)
            pdf.load()
            root = pdf.tree.getroot().getchildren()[0]
            page_box = [float(x) for x in root.get("bbox")[1:-1].split(",")]
            tables, _ =\
            zip(
                *sorted(
                    [(p.bounds,p.area) for p in cascaded_union(
                        [box(*[float(x) for x in node.get("bbox")[1:-1].split(",")]) for node in root.iter() if node.tag == "LTRect"]
                    )],
                    key = lambda x : -x[1]
                )
            )
            X = page_box[2]
            Y = page_box[3]
            xf = 11.69/X
            yf = 8.27/Y
            t1, t2 = tables

            table_1_bbox = ":".join(map(str,(t1[0]*xf - 0.1, (Y - t1[3])*yf - 0.1, t1[2]*xf + 0.1, (Y - t1[1])*yf + 0.1)))
            table_2_bbox = ":".join(map(str,(t2[0]*xf - 0.1, (Y - t2[3])*yf - 0.1, t2[2]*xf + 0.1, (Y - t2[1])*yf + 0.1)))

            df1 =\
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_1_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )
            _JSON["table_1"] = df_to_json(df1)
            df2 = \
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_2_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            _JSON["table_2"] = df2.to_json(orient='index')
    except Exception as e:
            return('Error:'+str(e))
    
    return json.dumps(_JSON, ensure_ascii=False)

def parse_other(pf):
    text=""
    if pf[-3:].lower()=='pdf':
                text=get_pdf_text_2(join(PDF_DIR,pf))
    if pf[-3:].lower()=='rtf':
                text=os.popen('cd "'+PDF_DIR+'" && unrtf --text '+pf).read()
    
    return text

for i, r in fdf.iterrows():
    ticresponse=""
    textresponse=""
    try:
        ticresponse=parse_ticket(join(PDF_DIR,r.filename))
    except Exception as e:
        ticresponse='Error:'+str(e)
    try:
        textresponse=parse_other(r.filename)
    except Exception as e:
        textresponse='Error:'+str(e)
    fdf.loc[i,"table_response"] = ticresponse
    fdf.loc[i,"text_response"] = textresponse
    fdf.loc[i,"hasid"]=hashlib.sha256(join(PDF_DIR,r.filename)).hexdigest()


In [5]:
with open('Final_Keyword_Analysis.pickle', 'rb') as handle:
    b=pickle.load(handle)
kxddf=b['keywordsX']


In [6]:
kxddf.loc[173]

fileclass                                           N5
filetype                                        TICKET
purpose                                 CLASSIFICATION
notification_type                 N5 - REJECTED CLAIMS
keyword              [ejecucion de titulos judiciales]
bias                                                N3
Name: 173, dtype: object

In [7]:
kxddf.drop(173,inplace=True)


In [8]:
fgs=fdf.groupby('filegroup')

len(fgs.groups)

144

In [9]:
fgs=fdf.groupby('filegroup')

fgdf=pd.DataFrame(columns=['filegroup'])
i=0
fgdf['files']=np.empty((len(fgs.groups), 0)).tolist()
fgdf['filetypes']=np.empty((len(fgs.groups), 0)).tolist()
for k,v in fgs.groups.items():
    
    fgdf.loc[i,'filegroup']=k
    files=[]
    pcs=[]
    filetypes=[]
    for ind in v:
            files.append(fdf.loc[ind,'filename'])
            filetypes.append(fdf.loc[ind,'filetype'])
    fgdf.loc[i,'filetypes']=filetypes
    fgdf.loc[i,'files']=files
    i+=1
fgdf          

,filegroup,files,filetypes
0,20181020780474620180509074923,[2009_0001419_EJH_2018102078047462018050907492...,"[TICKET, OTHER]"
1,20181020774383020180508154739,[2006_0001322_ETJ_2018102077438302018050815473...,"[OTHER, TICKET, OTHER]"
2,20181020764864120180508115825,[2009_0000528_ENJ_2018102076486412018050811582...,"[OTHER, TICKET]"
3,20181020779247720180508213035,[2010_0001969_ETJ_2018102077924772018050821303...,"[TICKET, OTHER, OTHER]"
4,20181020780654120180509080026,[2011_0001387_1TJ_2018102078065412018050908002...,"[OTHER, OTHER, TICKET]"
5,20181020771576220180508141013,[2009_0000877_ENJ_2018102077157622018050814101...,"[TICKET, OTHER]"
6,20181020780398220180509074522,[2009_0000661_ENJ_2018102078039822018050907452...,"[OTHER, OTHER, TICKET]"
7,20181020786419320180509100115,[2015_0000803_ETJ_2018102078641932018050910011...,"[TICKET, OTHER]"
8,20181020756481720180508092801,[1993_0000327_ETJ_2018102075648172018050809280...,"[TICKET, OTHER, OTHER, OTHER, OTHER]"
9,20181020783665920180509091724,[2012_0000582_ETJ_2018102078366592018050909172...,"[TICKET, OTHER, OTHER]"


In [3]:
c=0
for i ,r in fdf[~(fdf['table_response'].str.contains('Error:'))&(fdf['filetype']=='TICKET')].iterrows():
    if r['table_response'][:5]!='Error':
        js=json.loads(r['table_response'])
        pf=r['filename'].split('.')[0]+'_'+''.join(js['table_1']['Documentos'][0][0].split()).split('(Principal)')[0]
        fgf=[''.join(x.split())for x in fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0] ]
        if pf in fgf:
            fl=fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0][fgf.index(pf)]
            fdf.loc[fdf['filename'].str.contains(fl),'filetype']="NOTIFICATION"
            c+=1
print c
fdf

In [11]:
kdf=kxddf
pdfclass={'N1':'ADMISSION','N2':'TRANSFER OF LEGAL REPRESENTATION PERMITTED',
           'N3':'ENFORCEMENT ORDERS','N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS','N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING','N8':'ASSET INQUIRY',
           'N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT'}

In [15]:
kxddf.loc[173]

KeyError: u'the label [173] is not in the [index]'

In [4]:
for i,r in fdf.iterrows():
    try:
        newtext =unicode(textract.process(join(PDF_DIR,r['filename'])),'utf-8')
        newtext=(''.join((c for c in unicodedata.normalize('NFD', newtext) if unicodedata.category(c) != 'Mn'))).lower()
        rem=''
        paratlist=['MODO DE IMPUGNACION:'.lower(),'mode d\'impugnacio','recurso de repelacion','recurs de reposicio','recurso de reposicion','recurso de apelacion']
        if (r['filetype']=='NOTIFICATION') :
            for parat in paratlist:
                if (parat.lower() in newtext) :
                    rem=re.split(r'\s(?=(?:y firmo|y firma))',newtext.split(parat.lower())[-1],1)[0]
            newtext=newtext.replace(rem,'')
    except Exception as e:
        newtext='Error:'+str(e)
    fdf.loc[i,'text_response']=newtext
import re
kdf["document_list"]=np.empty((len(kdf), 0)).tolist()
fdf["keywords"]=fdf["filename"].apply(lambda x:{})

notextset=set()
for j,row in fdf.iterrows():

    if j%595==0:
        print j
    for i,kdrow in kdf.iterrows():
        text=''.join(row['text_response'].split())
#         
        if row['text_response'][:5]!='Error':
            
            f=True
            for k in kdrow['keyword']:
#                 if not re.search(k, row['text_response'], re.IGNORECASE):
                
#                 print(unidecode.unidecode(k).lower(),find_near_matches(unidecode.unidecode(k).lower(), text, max_l_dist=1))
#                 match=[]
#                 try:
#                     match=find_near_matches(str(unidecode.unidecode(k)).lower(), text, max_l_dist=2)
#                 except Exception as e:
#                     notextset.add(j)
#                 if len(match)==0:
#                     f=False
                if not (''.join(unidecode.unidecode(k).split()).lower() in text):
                    f=False
            if f and (kdrow['filetype']==row['filetype'])and (kdrow['purpose']=='CLASSIFICATION'):
                kdf.loc[i,"document_list"].append(row['hasid'])
                
                if kdrow['fileclass'] in fdf.loc[j,'keywords'].keys():
                    fdf.loc[j,'keywords'][kdrow['fileclass']].append(unidecode.unidecode(str(kdrow['keyword'])))
                else:
                    fdf.loc[j,'keywords'][kdrow['fileclass']]=list()
                    fdf.loc[j,'keywords'][kdrow['fileclass']].append(unidecode.unidecode(str(kdrow['keyword'])))
fdf

In [43]:
# dokydf=fdf.copy()[['filename','filetype','keywords']]
# keyrpodf=kdf.copy()
# for idf,rowdf in fdf.iterrows():
#     keywords=[]
#     keywordclass=[]
#     for k,v in rowdf['keywords'].items():
#         if 'key '+k in dokydf.columns:
#                 dokydf.loc[idf,'key '+k].append(v)
#         else:
#                 dokydf['key '+k]=np.empty((len(dokydf), 0)).tolist()
#                 dokydf.loc[idf,'key '+k].append(v)
#     for ki, kr in kdf.iterrows():
#             if rowdf['hasid'] in kr['document_list'] :
#                 if kr['fileclass'] in dokydf.columns:
#                     dokydf.loc[idf,kr['fileclass']]+=1
#                 else:
#                     dokydf[kr['fileclass']]=0
#                     dokydf.loc[idf,kr['fileclass']]=1
#                 if 'Filename '+ rowdf['fileclass'] in keyrpodf.columns:
#                     keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])
#                 else:
#                     keyrpodf['Filename '+ rowdf['fileclass']]=np.empty((len(keyrpodf), 0)).tolist()
#                     keyrpodf.loc[ki,'Filename '+ rowdf['fileclass']].append(rowdf['filename'])

KeyError: 'fileclass'

In [13]:
ddf=fdf.copy()[['filename','filegroup','hasid','filetype']]
for idf,rowdf in ddf.iterrows():
    for ki, kr in kxddf.iterrows():
        if rowdf['hasid'] in kr['document_list'] :
            c=1
            if not kr['fileclass']==kr['bias'] :
#                 c=kr[kr['file_class']]
                c=0
            if kr['fileclass'] in ddf.columns:
                ddf.loc[idf,kr['fileclass']]+=c
            else:
                ddf[kr['fileclass']]=0
                ddf.loc[idf,kr['fileclass']]=c
print("Part2")

for i,row in ddf.iterrows():
    keyscore=dict(row[['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10']])
    predicted=max(keyscore, key=keyscore.get)
    if 'N5'in row.to_dict().keys() and row['N5']>0:
        ddf.loc[i,'predicted_class']='N5'
    elif row['N2']>0 and row['N4']>0:
        ddf.loc[i,'predicted_class']='N2+N4'
    elif row['N1']>0 and(row['N10']>0 or row['N9']>0):
        ddf.loc[i,'predicted_class']='N1'
    elif row['N10']>0 and (row['N2']>0 or row['N6']>0):
        ddf.loc[i,'predicted_class']='N2+N4'
    elif keyscore[predicted]>0:
        ddf.loc[i,'predicted_class']=predicted
ddf

Part2


/usr/local/lib/python2.7/dist-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


,filename,filegroup,hasid,filetype,N4,N2,N8,N9,N10,N1,N6,N3,N5,predicted_class
0,2010_0000109_ETJ_20181020747526520180507164008...,20181020747526520180507164008,4c9a6a187d0702f0806304d7c7b48cdac3ff3954e37840...,TICKET,0,0,0,0,0,0,0,0,0,NaN
1,2010_0000109_ETJ_20181020747526520180507164008...,20181020747526520180507164008,66e243a5071d67b1be95841fafa7ebf39bf199ad5a0dac...,NOTIFICATION,3,0,0,0,0,0,0,0,0,N4
2,2008_0001402_ETJ_20181020752160320180508074031...,20181020752160320180508074031,cc301c15208149fd9cea282e9b640b2a2654fa84e2c910...,TICKET,0,0,0,0,0,0,0,0,0,NaN
3,2008_0001402_ETJ_20181020752160320180508074031...,20181020752160320180508074031,07c1d98461531a3179ad2eca8681e036a1f7b6c543c3cb...,NOTIFICATION,0,0,0,0,0,0,0,0,0,NaN
4,2010_0000009_ASE_20181020752218820180508074341...,20181020752218820180508074341,744ed3dbc76f106bd7e084d00dd8eb13cc493a3cbf5059...,OTHER,0,0,0,0,0,0,0,0,0,NaN
5,2010_0000009_ASE_20181020752218820180508074341...,20181020752218820180508074341,27df79d56c166f28d7ad10eb4a6bcd80166815bb2847ab...,TICKET,0,0,0,0,0,0,0,0,0,NaN
6,2010_0002296_ETJ_20181020753436520180508082849...,20181020753436520180508082849,f70c5688ae09a0ad59137039beca35ad956d1762410f9d...,NOTIFICATION,0,0,0,0,0,0,0,0,0,NaN
7,2010_0002296_ETJ_20181020753436520180508082849...,20181020753436520180508082849,ad37f70a8f03af66934c99007912bb95ad7ad457daef2d...,TICKET,0,0,0,0,0,0,0,0,0,NaN
8,2009_0001509_ENJ_20181020753595020180508083331...,20181020753595020180508083331,4499b0ab69b100bb02f4686aac7bc0c4899b81b0ad2887...,TICKET,0,0,0,0,0,0,0,0,0,NaN
9,2009_0001509_ENJ_20181020753595020180508083331...,20181020753595020180508083331,ef5fd26f65277d73b64d5d20033d3b8fbedfe28234418e...,NOTIFICATION,0,0,0,0,0,0,0,0,0,NaN


In [14]:
fgs=ddf.groupby('filegroup')
i=0
fgdf['predicted_classes']=np.empty((len(fgdf), 0)).tolist()
for k,v in fgs.groups.items():
    fgdf.loc[i,'filegroup']=k
    pcs=[]
    for ind in v:
            pcs.append(ddf.loc[ind,'predicted_class'])
    fgdf.loc[i,'predicted_classes']+=pcs
    i+=1

# y_actufg=[]
y_predfg=[]
conflic=[]
for k,v in fgs.groups.items():
#     y_actufg.append(ddf.loc[v[0],'fileclass'])
    s=set()
    e='_alpha'
    for i in v:
        s.add(ddf.loc[i,'predicted_class'])
#     s={x for x in s if x==x}
    if(len(s)==1):
        if list(s)[0]==list(s)[0]:
            e=list(s)[0]
            y_predfg.append(list(s)[0])
    else:
        
        if(len({x for x in s if x==x})>1):
            e='Conflict'
             
        else:
            
            for x in list(s):
                if x==x:
                    e=x
    if e!='_alpha' and e!='Conflict':
        fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
    elif e=='Conflict':
        for j in v:
            if ddf.loc[j,'filetype']=='TICKET':
                e=ddf.loc[j,'predicted_class']
                fgdf.loc[(fgdf['filegroup'] == k),'group_predicted_class']=e
#     y_predfg.append(e)        
        
fgdf       

,filegroup,files,filetypes,predicted_classes,group_predicted_class
0,20181020780474620180509074923,[2009_0001419_EJH_2018102078047462018050907492...,"[TICKET, OTHER]","[nan, nan]",NaN
1,20181020774383020180508154739,[2006_0001322_ETJ_2018102077438302018050815473...,"[OTHER, TICKET, OTHER]","[nan, nan, nan]",NaN
2,20181020764864120180508115825,[2009_0000528_ENJ_2018102076486412018050811582...,"[OTHER, TICKET]","[nan, nan]",NaN
3,20181020779247720180508213035,[2010_0001969_ETJ_2018102077924772018050821303...,"[TICKET, OTHER, OTHER]","[nan, nan, nan]",NaN
4,20181020780654120180509080026,[2011_0001387_1TJ_2018102078065412018050908002...,"[OTHER, OTHER, TICKET]","[nan, nan, nan]",NaN
5,20181020771576220180508141013,[2009_0000877_ENJ_2018102077157622018050814101...,"[TICKET, OTHER]","[N8, N8]",N8
6,20181020780398220180509074522,[2009_0000661_ENJ_2018102078039822018050907452...,"[OTHER, OTHER, TICKET]","[nan, nan, nan]",NaN
7,20181020786419320180509100115,[2015_0000803_ETJ_2018102078641932018050910011...,"[TICKET, OTHER]","[nan, N10]",N10
8,20181020756481720180508092801,[1993_0000327_ETJ_2018102075648172018050809280...,"[TICKET, OTHER, OTHER, OTHER, OTHER]","[nan, nan, nan, nan, nan]",NaN
9,20181020783665920180509091724,[2012_0000582_ETJ_2018102078366592018050909172...,"[TICKET, OTHER, OTHER]","[nan, nan, nan]",NaN


In [15]:
d={k:0 for k in list(set(x for x in fgdf['group_predicted_class'].values.tolist() if x==x))}
c=0
for x in fgdf['group_predicted_class'].values.tolist():
    if x==x:
        d[x]+=1
        c+=1
print d ,c

{'N10': 2, 'N8': 11, 'N9': 1, 'N2': 9, 'N3': 1, 'N4': 4, 'N6': 6} 34


In [16]:
a = {'documents':fdf,'filegroup':fgdf}
with open(PDF_DIR+'/FinalTest.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
procedures={'CAM':'Cambiario','COG':'Cognicion','CON':'Concurso Acreedores','EJE':'Juicio Ejecutivo','ETJ':'EJECUCION DE TITULOS JUDICIAL','ETJH':'ETJ Continuacion Hipotecario','ETN':'EJECUCION DE TITULOS NO JUDICIAL',\
            'HIP':'EJECUCION Hipotecaria','MCU':'Menor Cuantia','MON':'Monitorio','ORD':'‘Concurso Ordinario','TDO':'Terceria De Domino','TMD':'Terceria De Mejor Derecho','VER':'Verbal','PEN':'Penal','RAP':'Recurso De Apelacion',\
            'PTC':'Pieza Tasacion Costas','PSO':'Pieza Seperada Oposicion','AUX':'Auxilio Nacional','CNJ':'Cosignacion Judicial','RCS':'Recurso De Casacion','INC':'Incidente Concursal','MC':'Medidas Cautelares','CN':'Conciliacion'}
numspan={"1":"uno","2":"dos","3":"tres","4":"cuatro","5":"cinco","6":"seis","7":"siete","8":"ocho","9":"nueve","10":"diez","11":"once","12":"doce","13":"trece","14":"catorce","15":"quince","16":"dieciseis","17" :"diecisiete","18":"dieciocho","19":"diecinueve","20" : "veinte","21" : "veintiuno","22"  :"veintidós","23" : "veintitrés","24" : "veinticuatro","25" : "veinticinco","26":"veintiséis","27" :"veintisiete","28": "veintiocho","29"  :"veintinueve","30" : "treinta"}
numcat={"1":"un","2":"dos","3":"tres","4":"quatre","5":"cinc","6":"sis","7":"set","8":"vuit","9":"nou","10":"deu","11":"onze","12":"dotze","13":"tretze","14":"catorze","15":"quinze","16":"setze","17" :"disset","18":"divuit","19":"dinou","20" : "vint"}

In [18]:
extKeywords=kdf[kdf['purpose']=='EXTRACTION']
extKeywords['notification_type']=extKeywords['notification_type'].apply(lambda x : x.split('-')[1])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
c=0
fgdf['Numlist']=[[] for _ in range(len(fgdf))]
fgdf['MIN DAYS']=''
c=0
for fi,fr in fgdf.iterrows():
    auto=""
    match=None
    match1=set()
    match2=set()
    for i,r in fdf[(fdf['filetype']=='NOTIFICATION')&(fdf['filename'].isin(fr['files']))].iterrows():
        s= r['text_response']
        if len(s.strip())>0:
            s=unidecode.unidecode('\n'.join(list(filter(None,s.split('\n')))).lower())

            fgdf.loc[fi,'Court']=s.split('\n')[0]
            if 'procurador' in s.lower():
                fgdf.loc[fi,'Solictor']=list(filter(None,s[s.lower().index('procurador')+10:].split('\n')))[0]


            ptype=""

            for x in procedures.values():

                if x.lower() in ' '.join(s.split()):

                    fgdf.loc[fi,'Procedure_Type']=x
                    ptype=x
                    break
            if ptype!="":
                if fr['filegroup']=='INADMISION25':
                            print ' YESSS'
                if 'procedimiento' in s.lower():
                    match=re.search(r'(\d{1,20}/\d{4})',s.split('procedimiento')[1])

                else:
                    s1=''.join(s.split())
                    match1=set(re.findall(r'(\d{1,20}/\d{4})',s1))
                    match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s1)}

            if not match is None :
                    auto=match.group(0)
            elif not (match1-match2) is None :
                    auto=list(match1-match2)[0] if len(list(match1-match2))>0 else ''
    if(fr['group_predicted_class']=='N1')or(fr['group_predicted_class']=='N3')or(fr['group_predicted_class']=='N4'):
        for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('AMOUNT'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']==kr['filetype'])].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        tex=' '.join(r['text_response'].split())

                        fgdf.loc[fi,'Amount']=tex[tex.index(k1)+len(k1):].split()[0]
    if((fr['group_predicted_class']=='N9')or(fr['group_predicted_class']=='N10')):
#         for ki,kr in extKeywords[(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=' '.join(unidecode.unidecode(r['text_response']).split())
    #           
                nls=[]
                nlc=[]
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    
                    k1='dias'
                    
                    if  k1 in t.lower():
                          
                        nls+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dias' in x ]
                    elif  'dies' in t.lower():
                          
                        nlc+=  [t.lower().split()[i-1] for i, x in enumerate(t.lower().split()) if 'dies' in x ]
                    else:
                            f=False
                    if f :
                        min=100
                        ls=[]
                        if len(nls)>0:
                            fgdf.at[fi,'Numlist']=nls
                            ls=nls
                            numbers=numspan
                        elif len(nlc)>0:
                            fgdf.at[fi,'Numlist']=nlc
                            ls=nlc
                            numbers=numcat
                        for num in ls:
                            if num in numbers.values():
                                n=int([x for x in numbers.keys() if numbers[x]==num ][0])
                                if n < min:
                                    min=n
                            elif num in numbers.keys():
                                if int(num) < min:
                                    min=int(num)
                            else:
                                print num
                        if min!=100:
                            fgdf.loc[fi,'Time Frame']=min
                        
    if (fr['group_predicted_class']=='N5')|(fr['group_predicted_class']=='N6')|(fr['group_predicted_class']=='N9'):
        

        for ki,kr in extKeywords[~(extKeywords['notification_type'].str.contains('TIME FRAME'))&(extKeywords['fileclass']==fr['group_predicted_class'])].iterrows():
            for i,r in fdf[(fdf['filename'].isin(fr['files']))&(fdf['filetype']=='NOTIFICATION')].iterrows():
                text=''.join(r['text_response'].split())
    #         
                if r['text_response'][:5]!='Error':
                    
                    f=True
                    t=text
                    for k in kr['keyword']:
                        k1=''.join(unidecode.unidecode(k).split()).lower()
                        if  k1 in t:
                            t=t[t.index(k1)+len(k1):]
                        else:
                            f=False
                    k1=' '.join(kr['keyword'][0].split())    
                    if f :
                        fgdf.loc[fi,fr['group_predicted_class']+'-Extraction']=unidecode.unidecode(str([dict({kr['notification_type']:unidecode.unidecode(str(kr['keyword']))})]))
                        if fr['group_predicted_class']=='N5':
                            fgdf.loc[fi,'MIN DAYS']=5
    for i,r in fdf[(fdf['filetype']=='TICKET')&(fdf['filename'].isin(fr['files']))].iterrows():
        
            if r.table_response[:5]!='Error':
                try:
                    js= json.loads(unidecode.unidecode( json.dumps(json.loads(r.table_response), ensure_ascii=False)))
                except Exception as e:
                    js=json.loads(r.table_response)
                table2=json.loads(js['table_2'])
#                 print(r['filename'],js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]],table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]])
                c+=1
                fgdf.loc[fi,"Document date"]=js["table_1"][[x for x in js["table_1"].keys() if 'Fecha' in x][0]]
                fgdf.loc[fi,"Stamp date"]=table2['1'][[x for x in table2['1'].keys() if 'Fecha' in x][0]]
#                 print js
                
                
                try:
                    fgdf.loc[fi,'Solictor']=js["table_1"]['Destinatarios']['_value']
                    fgdf.loc[fi,'Court']=js["table_1"]['Remitente'][[x for x in js["table_1"]['Remitente'].keys() if str(x)[:6].lower()=='organo'][0]]

                except Exception as e:
                    c+=1
                auto=""
                try:
                    s=json.loads(unidecode.unidecode(r['table_response']))['table_1']['Datos del mensaje']['Procedimiento destino']

                except Exception as e:
                    s=str( json.loads(r['table_response'])['table_1']['Datos del mensaje'])
        #             print str(e)
                match1=set(re.findall(r'(\d{1,20}/\d{4})',s))
                match2={'/'.join(x.split('/')[1:])for x in re.findall(r'(\d{1,2}/\d{1,2}/\d{4})',s)}
                
                f=False
                for x in procedures.keys():
                    if x.lower() in  s.lower():
                        f=True
                        proced= procedures[x]
                if f:
                    fgdf.loc[fi,'Procedure_Type']=proced
                if not (match1-match2) is None :
                    auto=list(match1-match2)[0]
                    

    fgdf.loc[fi,'Auto']=auto
            
print c
fgdf

130


,filegroup,files,filetypes,predicted_classes,group_predicted_class,Numlist,MIN DAYS,Court,Solictor,Procedure_Type,Document date,Stamp date,Auto,Time Frame,Amount,N9-Extraction,N6-Extraction
0,20181020780474620180509074923,[2009_0001419_EJH_2018102078047462018050907492...,"[TICKET, OTHER]","[nan, nan]",NaN,[],,"JDO. PRIMERA INSTANCIA N. 2 de Logrono, La Rio...","GOMEZ DEL RIO, BLANCA LAURA [23]",Juicio Ejecutivo,09/05/2018 07:49,09/05/2018 08:12,0001419/2009,NaN,NaN,NaN,NaN
1,20181020774383020180508154739,[2006_0001322_ETJ_2018102077438302018050815473...,"[OTHER, TICKET, OTHER]","[nan, nan, nan]",NaN,[],,JDO. PRIMERA INSTANCIA N. 2 de Palmas de Gran ...,"HERNANDEZ PENATE, JOSE LORENZO [318]",Juicio Ejecutivo,08/05/2018 15:47,09/05/2018 08:12,0001322/2006,NaN,NaN,NaN,NaN
2,20181020764864120180508115825,[2009_0000528_ENJ_2018102076486412018050811582...,"[OTHER, TICKET]","[nan, nan]",NaN,[],,"JDO. PRIMERA INSTANCIA N. 15 de Sevilla, Sevil...","MALAGON LOYO, SILVIA [2058]",Juicio Ejecutivo,08/05/2018 11:58,09/05/2018 08:12,0000528/2009,NaN,NaN,NaN,NaN
3,20181020779247720180508213035,[2010_0001969_ETJ_2018102077924772018050821303...,"[TICKET, OTHER, OTHER]","[nan, nan, nan]",NaN,[],,JDO. PRIMERA INSTANCIA N. 5 de Alcala de Henar...,"ALCOCER ANTON, DOLORES [783]",‘Concurso Ordinario,08/05/2018 21:30,09/05/2018 08:18,0001969/2010,NaN,NaN,NaN,NaN
4,20181020780654120180509080026,[2011_0001387_1TJ_2018102078065412018050908002...,"[OTHER, OTHER, TICKET]","[nan, nan, nan]",NaN,[],,"JUTJAT. PRIMERA INSTANCIA N. 1 de Valencia, Va...","ALCOCER ANTON, DOLORES [783]",Concurso Acreedores,09/05/2018 08:00,09/05/2018 08:20,001387/2011,NaN,NaN,NaN,NaN
5,20181020771576220180508141013,[2009_0000877_ENJ_2018102077157622018050814101...,"[TICKET, OTHER]","[N8, N8]",N8,[],,JDO. 1A. INST. E INSTRUCCION N. 5 de Puerto de...,"GARCIA ABASCAL, SUSANA [721]",Juicio Ejecutivo,08/05/2018 14:09,09/05/2018 08:24,0000877/2009,NaN,NaN,NaN,NaN
6,20181020780398220180509074522,[2009_0000661_ENJ_2018102078039822018050907452...,"[OTHER, OTHER, TICKET]","[nan, nan, nan]",NaN,[],,JUTJAT. PRIMERA INSTANCIA N. 1 de Alicante/Ala...,"ALCOCER ANTON, DOLORES [783]",EJECUCION DE TITULOS NO JUDICIAL,09/05/2018 07:45,09/05/2018 08:18,661/2009,NaN,NaN,NaN,NaN
7,20181020786419320180509100115,[2015_0000803_ETJ_2018102078641932018050910011...,"[TICKET, OTHER]","[nan, N10]",N10,[cinco],,"JDO. 1A. INST. E INSTRUCCION N. 1 de Motril, G...","ALCOCER ANTON, DOLORES [783]",Juicio Ejecutivo,09/05/2018 10:01,09/05/2018 10:47,0000803/2015,5.0,NaN,NaN,NaN
8,20181020756481720180508092801,[1993_0000327_ETJ_2018102075648172018050809280...,"[TICKET, OTHER, OTHER, OTHER, OTHER]","[nan, nan, nan, nan, nan]",NaN,[],,JDO. 1A. INST. E INSTRUCCION N. 3 de Arcos de ...,"SEVILLA RAMIREZ, JOSE MARIA [33]",Juicio Ejecutivo,08/05/2018 09:27,09/05/2018 08:24,0000327/1993,NaN,NaN,NaN,NaN
9,20181020783665920180509091724,[2012_0000582_ETJ_2018102078366592018050909172...,"[TICKET, OTHER, OTHER]","[nan, nan, nan]",NaN,[],,JDO. 1A. INST. E INSTRUCCION N. 1 de Granadill...,"MALAGON LOYO, SILVIA [2058]",Juicio Ejecutivo,09/05/2018 09:17,09/05/2018 12:26,0000582/2012,NaN,NaN,NaN,NaN


In [20]:
# # fgdf['filegroup'].values.tolist()
# fgdf['Keywords']=[[] for _ in range(len(fgdf))]
# for i , r in fgdf.iterrows():
#     for f in r['files']:
#         for k,v in fdf.loc[fdf['filename']==f]['keywords'].values[0].items():
            
#             if len(fgdf.loc[i,'Keywords'])>0 and  k in fgdf.loc[i,'Keywords'][0].keys() :
#                 fgdf.loc[i,'Keywords'][0][k]+=v
#             else:
#                 fgdf.loc[i,'Keywords']=[{k:v}]
# fgdf['Keywords']=fgdf['Keywords'].apply(lambda x :x[0] if len(x)>0 else {})
# fgdf
len(ddf[~ddf['predicted_class'].isnull()])

43

In [21]:
a = {'fg':fgdf}
with open(PDF_DIR+'/TestFileGroup.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
writer = pd.ExcelWriter(PDF_DIR+'/'+str(time.time())+'File_Details.xlsx', engine='openpyxl')
fdf[['filename','filegroup','filetype','keywords']].to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter(PDF_DIR+'/'+str(time.time())+'File_Group_Result.xlsx', engine='openpyxl')
fgdf.to_excel(writer,'Sheet1')
writer.save()